In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing
from scipy.stats import shapiro, mannwhitneyu

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()
df = pd.DataFrame({'a' : [1, 2, 3], 'target' : [3, 3, 3]})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# !pip install lightgbm
# !pip install xgboost
# !pip install catboost

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [ ]:
train = pd.read_csv('/content/sample_data/black_friday_train.csv')
train.head()

In [ ]:
sns.distplot(train['Purchase'])
plt.show()

In [ ]:
X = train.drop(["Purchase"], axis=1) #убираем колонку с названием Purchase
y = train["Purchase"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

> ## Random Forest


In [ ]:
rf = RandomForestRegressor(
                      n_estimators = 700, # Кол-во деревьев
                      max_depth=9,        # Глубина
                      random_state=42,
                      n_jobs = -1)         #Мултипоточность


In [ ]:
rf.fit(x_train, y_train)

# y_rf_pred = rf.predict(x_test)

# rmse(10**y_test, 10**y_rf_pred)
# r2_score(10**y_test, 10**y_rf_pred)

> ## Boosting

* Gradient Boosting

In [ ]:
gbr = GradientBoostingRegressor(
      n_estimators=260,
      learning_rate=0.05,
      max_depth=4,
      loss='huber',
      random_state =42)

In [ ]:
gbr.fit(x_train, y_train)
# y_gb_pred = gbr.predict(x_test)

# rmse(10**y_test, 10**y_gb_pred)
# r2_score(10**y_test, 10**y_gb_pred)

* Light GBM

In [ ]:
lightgbm = LGBMRegressor(
          objective='regression',
          learning_rate=0.04,
          n_estimators=250,
          max_depth=6,
          n_jobs=-1,
          verbose=-1, #progress messages are printed or not
          )

In [ ]:
lightgbm.fit(x_train, y_train)

# y_pred_lgb = lightgbm.predict(x_test)

# rmse(10**y_test, 10**y_pred_lgb)
# r2_score(10**y_test, 10**y_pred_lgb)

* XGBoost


In [ ]:
xgb = XGBRegressor(
                    learning_rate=0.05,
                    n_estimators=600,
                    max_depth=4,
                    objective='reg:squarederror',
                    nthread=-1,
                    n_jobs=-1,
                    )

In [ ]:
xgb.fit(x_train, y_train)

# y_pred_xgb = xgb.predict(x_test)

# rmse(10**y_test, 10**y_pred_xgb)
# r2_score(10**y_test, 10**y_pred_xgb)

> # Stacking

In [ ]:
test = pd.read_csv('/content/sample_data/black_friday_test.csv')

In [ ]:
stack_gen = StackingCVRegressor(regressors=(
                                            rf,
                                            gbr,
                                            xgb,
                                            lightgbm,
                                           ),
                                meta_regressor=xgb,
                                use_features_in_secondary=True)

In [ ]:
stack_gen.fit(X.values, y)

In [ ]:
def blend_models_predict(X_in):
    return (
            (0.1 * rf.predict(X_in)) + \
            (0.2 * gbr.predict(X_in)) + \
            (0.1 * lightgbm.predict(X_in)) + \
            (0.2 * xgb.predict(X_in)) + \
            (0.4 * stack_gen.predict(X_in.values)))

In [ ]:
test_preds = blend_models_predict(test)

In [ ]:
submission = pd.read_csv('/content/sample_data/sample_submission.csv')

submission['Purchase'] = test_preds
submission.to_csv('my_update14_lgbm_cat.csv', index=False)
submission